In [ ]:
import tensorflow as tf
import math
print('TensorFlow version: ' + tf.__version__)

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data as mnist_data
mnist = mnist_data.read_data_sets("../MNIST_data", one_hot=True, reshape=True, validation_size=0)

x_train = mnist.train.images # we will not be using these to feed in data
y_train = mnist.train.labels # instead we will be using next_batch function to train in batches
x_test = mnist.test.images
y_test = mnist.test.labels

print ('We have '+str(x_train.shape[0])+' training examples in dataset')

In [ ]:
TUTORIAL_NAME = 'Tutorial3a'
MODEL_NAME = 'convnetTFonAndroid'
SAVED_MODEL_PATH = '../' + TUTORIAL_NAME+'_Saved_model/'

In [ ]:
BATCH_SIZE = 100
KEEP_PROB = 0.75

LEARNING_RATE_MAX = 1e-1    #1e-2
LEARNING_RATE_MIN = 1e-1    #1e-4
TRAIN_STEPS = 600           #600 * 2

for i in range(TRAIN_STEPS+1):

    up = -math.log(LEARNING_RATE_MAX,10)
    lp = -math.log(LEARNING_RATE_MIN,10)
    
    LEARNING_RATE = LEARNING_RATE_MIN + (LEARNING_RATE_MAX - LEARNING_RATE_MIN)* math.pow(10,(up - lp)*((i)/(TRAIN_STEPS+1e-9)))
    if i%(TRAIN_STEPS/10)==0:
        print LEARNING_RATE


In [ ]:
keepProb = tf.placeholder(tf.float32)
lRate = tf.placeholder(tf.float32)

W1 = tf.Variable(tf.truncated_normal([6, 6, 1, 6], stddev=0.1)) 
B1 = tf.Variable(tf.constant(0.1, tf.float32, [6]))
W2 = tf.Variable(tf.truncated_normal([5, 5, 6, 12], stddev=0.1))
B2 = tf.Variable(tf.constant(0.1, tf.float32, [12]))
W3 = tf.Variable(tf.truncated_normal([4, 4, 12, 24], stddev=0.1))
B3 = tf.Variable(tf.constant(0.1, tf.float32, [24]))

W4 = tf.Variable(tf.truncated_normal([7 * 7 * 24, 200], stddev=0.1))
B4 = tf.Variable(tf.constant(0.1, tf.float32, [200]))
W5 = tf.Variable(tf.truncated_normal([200, 10], stddev=0.1))
B5 = tf.Variable(tf.constant(0.1, tf.float32, [10]))

# The model
X = tf.placeholder(tf.float32, [None, 28*28], name='modelInput')
X_image = tf.reshape(X, [-1, 28, 28, 1])
Y_ = tf.placeholder(tf.float32, [None, 10])

Y1 = tf.nn.relu(tf.nn.conv2d(X_image, W1, strides=[1, 1, 1, 1], padding='SAME') + B1)
Y2 = tf.nn.relu(tf.nn.conv2d(Y1, W2, strides=[1, 2, 2, 1], padding='SAME') + B2)
Y3 = tf.nn.relu(tf.nn.conv2d(Y2, W3, strides=[1, 2, 2, 1], padding='SAME') + B3)

YY = tf.reshape(Y3, shape=[-1, 7 * 7 * 24])

Y4 = tf.nn.relu(tf.matmul(YY, W4) + B4)
YY4 = tf.nn.dropout(Y4, keepProb)
Ylogits = tf.matmul(YY4, W5) + B5
Y = tf.nn.softmax(Ylogits)

#########
Ylogits = tf.matmul(Y4, W5) + B5
Y_inf = tf.nn.softmax(Ylogits, name='modelOutput')
#########

In [ ]:
cross_entropy = 100 * tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=Ylogits, labels=Y_))
correct_prediction = tf.equal(tf.argmax(Y, 1), tf.argmax(Y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
train_step = tf.train.AdamOptimizer(lRate).minimize(cross_entropy)

In [ ]:
tf.set_random_seed(0)

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
saver = tf.train.Saver()

In [ ]:
for i in range(TRAIN_STEPS+1):

    up = -math.log(LEARNING_RATE_MAX,10)
    lp = -math.log(LEARNING_RATE_MIN,10)
    
    LEARNING_RATE = LEARNING_RATE_MIN + (LEARNING_RATE_MAX - LEARNING_RATE_MIN)* math.pow(10,(up - lp)*((i)/(TRAIN_STEPS+1e-9)))
    
    batch_X, batch_Y = mnist.train.next_batch(BATCH_SIZE)
    
    sess.run(train_step, feed_dict={X: batch_X, Y_: batch_Y, lRate:LEARNING_RATE, keepProb:KEEP_PROB})
    if i%100 == 0:
        print('Latest learning rate is: '+str(LEARNING_RATE))
    if i%100 == 0:
        print('Training Step:' + str(i) 
              + '  Accuracy =  ' + str(sess.run(accuracy, feed_dict={X: x_test, Y_: y_test, keepProb:1.0})) 
              + '  Loss = ' + str(sess.run(cross_entropy, {X: x_test, Y_: y_test})) 
             )
    # uncomment this when learning rate is non-zero to save checkpoints
    #if i%100 == 0:
        #out = saver.save(sess, SAVED_MODEL_PATH + MODEL_NAME + '.ckpt', global_step=i)


In [ ]:
tf.train.write_graph(sess.graph_def, SAVED_MODEL_PATH , MODEL_NAME + '.pbtxt')
tf.train.write_graph(sess.graph_def, SAVED_MODEL_PATH , MODEL_NAME + '.pb',as_text=False)

In [ ]:
from tensorflow.python.tools import freeze_graph

# Freeze the graph
input_graph = SAVED_MODEL_PATH+MODEL_NAME+'.pb'
input_saver = ""
input_binary = True
input_checkpoint = SAVED_MODEL_PATH+MODEL_NAME+'.ckpt-1200'
output_node_names = 'modelOutput'
restore_op_name = 'save/restore_all'
filename_tensor_name = 'save/Const:0'
output_graph = SAVED_MODEL_PATH+'frozen_'+MODEL_NAME+'.pb'
clear_devices = True
initializer_nodes = ""
variable_names_blacklist = ""

freeze_graph.freeze_graph(
    input_graph,
    input_saver,
    input_binary,
    input_checkpoint,
    output_node_names,
    restore_op_name,
    filename_tensor_name,
    output_graph,
    clear_devices,
    initializer_nodes,
    variable_names_blacklist
)